<a href="https://colab.research.google.com/github/mallika100294/Mallika_CV/blob/master/Project_F_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Upgrade TensorFlow and Keras
# !pip install --upgrade tensorflow
# !pip install --upgrade keras

# Standard imports
import numpy as np
import pandas as pd
import csv
import argparse
import os
import json
import random
import math

# Keras imports
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Conv1D, MaxPooling1D, Bidirectional, LSTM, Dropout, GlobalMaxPooling1D
from keras.initializers import Constant
from keras import backend as K
import keras


from nltk.corpus import stopwords
import nltk
BASE_DIR = 'drive/My Drive/ECE542-ProjectF/'
DATA_DIR = BASE_DIR + 'data/'
GLOVE_DIR = 'drive/My Drive/ECE542-ProjectF/data/GloVe'
OUT_DIR = 'drive/My Drive/ECE542-ProjectF/output'
# TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100

ISEAR_SHUFFLE_SEED = 62
KAGGLE_SHUFFLE_SEED = 43

ISEAR_TEST_SPLIT = 0.1
ISEAR_VALIDATION_SPLIT = 0.1
KAGGLE_VALIDATION_SPLIT = 0.2


Using TensorFlow backend.


In [0]:
def split_data(data, labels, val_split, test_split, seed):
  # Shuffle our dataset. Using a known seed allows us to get the same result 
  # each time we run the script.
  indices = np.arange(data.shape[0])
  random.Random(seed).shuffle(indices)

  data = data[indices]
  labels = labels[indices]

  # Determine number of samples in train/val/test
  num_val = math.floor(data.shape[0] * val_split)
  num_test = math.floor(data.shape[0] * test_split)

  # Split the data into the test, val, and train subsets.
  x_test = data[:num_test]
  y_test = labels[:num_test]

  x_val = data[num_test:num_val + num_test]
  y_val = labels[num_test:num_val + num_test]

  x_train = data[num_val + num_test:]
  y_train = labels[num_val + num_test:]

  # Return a dictionary of the data with train/val/test and x/y.
  return {
      'train': {
          'x': x_train, 'y': y_train
      },
      'val': {
          'x': x_val, 'y': y_val
      },
      'test': {
          'x': x_test, 'y': y_test
      }
  }

def prec_m(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    pp = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = tp / (pp + K.epsilon())
    return precision

def recall_m(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    pp = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = tp / (pp + K.epsilon())
    return recall

def f1_m(y_true, y_pred):
    precision = prec_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision + recall + K.epsilon()))

In [0]:
# Load the ISEAR and Kaggle datasets and Tokenize the datasets
# Both datasets have the label as their first index and the text as their second

isear_text_data = []
isear_labels = []
kaggle_text_data = []
kaggle_labels = []

with open(('drive/My Drive/ECE542-ProjectF/data/isear/isear_clean.csv'), 'r') as f:
  reader = csv.reader(f)
  for row in reader:
    isear_text_data.append(row[1])
    isear_labels.append(int(row[0]))

with open(('drive/My Drive/ECE542-ProjectF/data/kaggle/kaggle_clean.csv'), 'r') as f:
  reader = csv.reader(f)
  for row in reader:
    kaggle_text_data.append(row[1])
    kaggle_labels.append(int(row[0]))


In [0]:
isear_text_data_DF = pd.DataFrame(isear_text_data)
isear_text_data_DF.columns =["text"]
isear_text_data_DF


,text
0,"During the period of falling in love, each tim..."
1,When I was involved in a traffic accident.
2,When I was driving home after several days of...
3,When I lost the person who meant the most to me.
4,The time I knocked a deer down - the sight of ...
...,...
7462,Two years back someone invited me to be the tu...
7463,I had taken the responsibility to do something...
7464,I was at home and I heard a loud sound of spit...
7465,I did not do the homework that the teacher had...


In [0]:
isear_text_data_DF["text"]
isear_text_data_DF["text"] = [entry.lower() for entry in isear_text_data_DF["text"]]

In [0]:
#nltk.download('stopwords')
stop = stopwords.words('english')
isear_text_data_DF['tweet_without_stopwords'] = isear_text_data_DF['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
isear_text_data_DF[['text','tweet_without_stopwords']]

,text,tweet_without_stopwords
0,"during the period of falling in love, each tim...","period falling love, time met especially met l..."
1,when i was involved in a traffic accident.,involved traffic accident.
2,when i was driving home after several days of...,"driving home several days hard work, motorist ..."
3,when i lost the person who meant the most to me.,lost person meant me.
4,the time i knocked a deer down - the sight of ...,time knocked deer - sight animal's injuries he...
...,...,...
7462,two years back someone invited me to be the tu...,two years back someone invited tutor grand-dau...
7463,i had taken the responsibility to do something...,taken responsibility something prepared it. ho...
7464,i was at home and i heard a loud sound of spit...,home heard loud sound spitting outside door. t...
7465,i did not do the homework that the teacher had...,homework teacher asked us do. scolded immediat...


In [0]:
## Tokenize ISEAR

# ISEAR
isear_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
isear_tokenizer.fit_on_texts(isear_text_data)
isear_seq = isear_tokenizer.texts_to_sequences(isear_text_data)
isear_index = isear_tokenizer.word_index
isear_data = pad_sequences(isear_seq, maxlen=MAX_SEQUENCE_LENGTH)
isear_labels = to_categorical(isear_labels)
print('ISEAR Data Shapes:')
print('Index size: ', len(isear_index))
print('Data tensor: ', isear_data.shape)
print('Label tensor: ', isear_labels.shape)

# Shuffle our dataset. Using a known seed allows us to get the same result 
# each time we run the script.
isear_split = split_data(
    isear_data, 
    isear_labels, 
    ISEAR_VALIDATION_SPLIT, 
    ISEAR_TEST_SPLIT, 
    ISEAR_SHUFFLE_SEED
  )

print('ISEAR Split shapes:')
for data_type, items in isear_split.items():
  print('{0:5s}:'.format(data_type.capitalize()))
  print('x: ', items['x'].shape, 'y: ', items['y'].shape)


In [0]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


In [0]:
data=pandas.read_csv('drive/My Drive/ECE542-ProjectF/data/isear/isear_clean.csv')
data=pandas.DataFrame(data)
data.columns=['label','text']

x_train, x_val, y_train, y_val = model_selection.train_test_split(data['text'], data['label'])



def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
  # fit the training dataset on the classifier
  classifier.fit(feature_vector_train, label)
  # predict the labels on validation dataset
  predictions = classifier.predict(feature_vector_valid)
  if is_neural_net:
    predictions = predictions.argmax(axis=-1)
  return metrics.accuracy_score(predictions, y_val)


tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,1), max_features=10000)
tfidf_vect_ngram.fit(data['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(x_train)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(x_val)


# label encode the target variable 
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.fit_transform(y_val)


accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, y_train, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)




SVM, N-Gram Vectors:  0.5757900374933048


In [0]:
data=pd.read_csv('drive/My Drive/ECE542-ProjectF/data/isear/isear_clean.csv')
data_DF=pd.DataFrame(data)
data_DF.columns=['label','text']
data_DF["text"] = [entry.lower() for entry in data_DF["text"]]
stop = stopwords.words('english')
data_DF['tweet_without_stopwords'] = data_DF['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data_DF[['label','tweet_without_stopwords']]

x_train, x_val, y_train, y_val = model_selection.train_test_split(data_DF['tweet_without_stopwords'], data_DF['label'])

def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
  # fit the training dataset on the classifier
  classifier.fit(feature_vector_train, label)
  # predict the labels on validation dataset
  predictions = classifier.predict(feature_vector_valid)

  if is_neural_net:
    predictions = predictions.argmax(axis=-1)
  return metrics.accuracy_score(predictions, y_val)

tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,1), max_features=10000)
tfidf_vect_ngram.fit(data_DF['tweet_without_stopwords'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(x_train)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(x_val)


# label encode the target variable 
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.fit_transform(y_val)


accuracy = train_model(svm.SVC(kernel='poly', degree=3, C=1.0), xtrain_tfidf_ngram, y_train, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)




SVM, N-Gram Vectors:  0.46009641135511514
